In [1]:
# import the library
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

In [2]:
# initiate spark

import findspark

findspark.init('/Users/arybressane/spark')

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext()
spark = SparkSession(sc)

In [3]:
# import spark libraries
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Problem definition

Apply regression models to predict the house pricing

# Load the data

In [4]:
# Load and parse the data file, converting it to a DataFrame.
data = spark.read.csv('data/default of credit card clients.csv', header = True, inferSchema = True)
data.show

<bound method DataFrame.show of DataFrame[ID: int, LIMIT_BAL: int, SEX: int, EDUCATION: int, MARRIAGE: int, AGE: int, PAY_0: int, PAY_2: int, PAY_3: int, PAY_4: int, PAY_5: int, PAY_6: int, BILL_AMT1: int, BILL_AMT2: int, BILL_AMT3: int, BILL_AMT4: int, BILL_AMT5: int, BILL_AMT6: int, PAY_AMT1: int, PAY_AMT2: int, PAY_AMT3: int, PAY_AMT4: int, PAY_AMT5: int, PAY_AMT6: int, default payment next month: int]>

# Feature Engineering 

In [5]:
# feature engineering
X_columns = data.columns[:-1]
y_column = data.columns[-1]

In [6]:
# Create the features column
vecAssembler = VectorAssembler(inputCols=X_columns, outputCol="features")
data = vecAssembler.transform(data)

# Split the data into training and test sets (80% held out for testing)
(trainingData, testData) = data.randomSplit([0.8, 0.2])

# Model Training

In [7]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol=y_column, numTrees=100)

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", y_column, "features").show(5)

rfModel = model.stages
print(rfModel)  # summary only

+----------+--------------------------+--------------------+
|prediction|default payment next month|            features|
+----------+--------------------------+--------------------+
|       0.0|                         0|[8.0,100000.0,2.0...|
|       0.0|                         0|[9.0,140000.0,2.0...|
|       0.0|                         1|[14.0,70000.0,1.0...|
|       0.0|                         0|[15.0,250000.0,1....|
|       0.0|                         0|[16.0,50000.0,2.0...|
+----------+--------------------------+--------------------+
only showing top 5 rows

[RandomForestClassificationModel (uid=RandomForestClassifier_42218352ba2efb86d0bb) with 100 trees]


# Model Evaluation

In [8]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol=y_column, predictionCol="prediction", metricName="weightedPrecision")
mae = evaluator.evaluate(predictions)
print("Precision on test data = %g" % mae)

evaluator = MulticlassClassificationEvaluator(labelCol=y_column, predictionCol="prediction", metricName="weightedRecall")
rmse = evaluator.evaluate(predictions)
print("Recall on test data = %g" % rmse)

rfModel = model.stages
print(rfModel)  # summary only

Precision on test data = 0.790518
Recall on test data = 0.808764
[RandomForestClassificationModel (uid=RandomForestClassifier_42218352ba2efb86d0bb) with 100 trees]
